In [83]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Blues, BuGn, viridis, Paired, plasma
from bokeh.models import FixedTicker, FactorRange,Color,CustomJS, HoverTool,CategoricalAxis, LabelSet, Label, ColumnDataSource, widgets,CategoricalColorMapper,LinearInterpolator, LinearColorMapper, LogColorMapper
from bokeh.models import (GMapPlot, GMapOptions, Range1d, PanTool, WheelZoomTool, BoxSelectTool, HoverTool,  ResetTool, ZoomInTool, ZoomOutTool)
from bokeh.layouts import row, column, widgetbox
from bokeh.models.glyphs import Patches, Line, Circle
from bokeh.charts import Histogram, output_file, show, Bar, color,  Scatter
from bokeh.resources import CDN
import sklearn
from sklearn.utils.validation import check_array
import random
import squarify
import pandas as pd

output_notebook()


%load_ext sql
%sql sqlite:///mit-poster.db
%sql ATTACH '../jupyterdemo/claims.db' as mem;


Loading BokehJS ...

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Done.


[]

In [68]:
from hcc import *


In [69]:
daniel

ID:1,DOB:1974-08-24 00:00:00

In [71]:
yr1_diags = %sql SELECT DISTINCT Diagnosis  From diagnoses  where PatientID = '0220F11E0B2EC004' and ClaimFromDate like '2008%';
yr2_diags = %sql SELECT DISTINCT Diagnosis  From diagnoses  where PatientID = '0220F11E0B2EC004' and ClaimFromDate like '2009%';

Done.
Done.


In [72]:
jane = Beneficiary(hicno='0220F11E0B2EC004', sex='female', dob='19480601')
jane_alt = Beneficiary(hicno='0220F11E0B2EC004', sex='male', dob='19480601')

#antonio.add_diagnosis(Diagnosis(antonio,"49320",ICDType.NINE))


for icd9 in yr1_diags:
    code = icd9[0]
    #print(type(code), code)
    jane.add_diagnosis(Diagnosis(jane,code, int(ICDType.NINE)))

for icd9 in yr1_diags + yr2_diags:
    code = icd9[0]
    #print(type(code), code)
    jane.add_diagnosis(Diagnosis(jane_alt,code, int(ICDType.NINE)))


            

In [73]:
beneficiary_has_hcc(jane, X)

print(score(jane,"community",Score), score(jane_alt,"community",Score))

Score 
------
17.213 Score
-----
21.09


In [98]:
score(jane,"community",Score)[0][0]

17.213

In [74]:
cvars= community_regression()
indicator(jane,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)

[('COPD_CARD_RESP_FAIL', 0.42), ('HCC111', 0.388), ('F65_69', 0.283), ('HCC75', 0.281), ('HCC23', 0.24), ('HCC134', 0.617), ('CHF_COPD', 0.255), ('CANCER_IMMUNE', 1.101), ('HCC115', 0.212), ('HCC2', 0.52), ('HCC170', 0.363), ('HCC21', 0.653), ('SEPSIS_CARD_RESP_FAIL', 0.634), ('HCC85', 0.361), ('HCC40', 0.376), ('CHF_RENAL', 0.201), ('HCC87', 0.283), ('HCC176', 0.555), ('HCC8', 2.425), ('HCC84', 0.326), ('HCC186', 1.032), ('HCC106', 1.313), ('HCC46', 1.078), ('HCC96', 0.276), ('HCC160', 1.071), ('HCC29', 0.313), ('HCC47', 0.306), ('DIABETES_CHF', 0.237), ('HCC17', 0.344), ('HCC188', 0.609), ('HCC39', 0.423)]

In [75]:
indicator(jane_alt,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)

[('HCC87', 0.283), ('HCC85', 0.361), ('SEPSIS_CARD_RESP_FAIL', 0.634), ('HCC157', 1.071), ('HCC23', 0.24), ('HCC170', 0.363), ('HCC39', 0.423), ('HCC80', 0.533), ('HCC111', 0.388), ('HCC82', 1.732), ('HCC96', 0.276), ('HCC46', 1.078), ('HCC186', 1.032), ('DIABETES_CHF', 0.237), ('HCC79', 0.252), ('HCC29', 0.313), ('HCC134', 0.617), ('CANCER_IMMUNE', 1.101), ('CHF_RENAL', 0.201), ('HCC40', 0.376), ('HCC47', 0.306), ('HCC33', 0.337), ('COPD_CARD_RESP_FAIL', 0.42), ('HCC58', 0.318), ('HCC176', 0.555), ('HCC78', 0.555), ('CHF_COPD', 0.255), ('HCC75', 0.281), ('HCC188', 0.609), ('HCC17', 0.344), ('HCC106', 1.313), ('HCC8', 2.425), ('HCC21', 0.653), ('HCC52', 0.343), ('M65_69', 0.309), ('HCC2', 0.52), ('HCC115', 0.212), ('HCC173', 0.379)]

In [76]:
len(indicator(jane,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef))



31

In [77]:
x = 0
y = 0
width = 40
height = 40
jane_coefs = indicator(jane,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)
jane_coefs = sorted(jane_coefs,key= lambda val:  val[1], reverse=True)
print(jane_coefs)
normed = squarify.normalize_sizes([value for name, value in jane_coefs], width, height)
rects = squarify.squarify(normed, x, y, width, height)
shapes = []
annotations = []
counter = 0
for r in rects:
    name, value = jane_coefs[counter]
    shapes.append(
        dict(
            x0 = r['x'] + r['dx']/2 ,
            y0 = r['y'] + r['dy']/2 , 
            width = r['dx'],
            height = r['dy'],
            color=random.choice(Paired[12]),
            text=name,
            score=value,
            x1 = r['x'],
            y1 = r['y']
        ) 
    )
    counter = counter + 1
 


r_data = ColumnDataSource(ColumnDataSource.from_df(pd.DataFrame.from_dict(shapes)))


q = figure(x_range=Range1d(x, width), y_range=Range1d(y, height))
q.rect( x='x0', y='y0', width='width', height='height', fill_color='color',
           line_alpha=0, source=r_data)
q.text(x='x1', y='y1', text='text', source=r_data, x_offset=3, y_offset=1
       , text_font_size='5pt')
q.xaxis.visible = False
q.yaxis.visible = False





[('HCC8', 2.425), ('HCC106', 1.313), ('CANCER_IMMUNE', 1.101), ('HCC46', 1.078), ('HCC160', 1.071), ('HCC186', 1.032), ('HCC21', 0.653), ('SEPSIS_CARD_RESP_FAIL', 0.634), ('HCC134', 0.617), ('HCC188', 0.609), ('HCC176', 0.555), ('HCC2', 0.52), ('HCC39', 0.423), ('COPD_CARD_RESP_FAIL', 0.42), ('HCC111', 0.388), ('HCC40', 0.376), ('HCC170', 0.363), ('HCC85', 0.361), ('HCC17', 0.344), ('HCC84', 0.326), ('HCC29', 0.313), ('HCC47', 0.306), ('HCC87', 0.283), ('F65_69', 0.283), ('HCC75', 0.281), ('HCC96', 0.276), ('CHF_COPD', 0.255), ('HCC23', 0.24), ('DIABETES_CHF', 0.237), ('HCC115', 0.212), ('CHF_RENAL', 0.201)]


In [79]:

janealt_coefs = indicator(jane_alt,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)
janealt_coefs = sorted(janealt_coefs,key= lambda val:  val[1], reverse=True)
normed_alt = squarify.normalize_sizes([value for name, value in janealt_coefs], width, height)
rects_alt = squarify.squarify(normed_alt, x, y, width, height)
shapes_alt = []

counter_alt = 0
for r in rects_alt:
    name, value = janealt_coefs[counter]
    shapes_alt.append(
        dict(
            x0 = r['x'] + r['dx']/2 ,
            y0 = r['y'] + r['dy']/2 , 
            width = r['dx'],
            height = r['dy'],
            color=random.choice(Paired[12]),
            text=name,
            score=value,
            x1 = r['x'],
            y1 = r['y']
        ) 
    )
    counter_alt = counter_alt + 1

z_data = ColumnDataSource(ColumnDataSource.from_df(pd.DataFrame.from_dict(shapes_alt)))


z = figure(x_range=Range1d(x, width), y_range=Range1d(y, height))
z.rect( x='x0', y='y0', width='width', height='height', fill_color='color',
           line_alpha=0, source=z_data)
z.text(x='x1', y='y1', text='text', source=z_data, x_offset=3, y_offset=1
       , text_font_size='5pt')
z.xaxis.visible = False
z.yaxis.visible = False






In [80]:
show(row(q,z))

In [87]:
cohort_study = %sql \
    SELECT d.PatientID, p.SEX, p.DOB \
        from diagnoses d join mem.patients p\
                on d.PatientID = p.PATIENT_ID\
        group by PatientID,p.SEX, p.DOB\
        having COUNT(*) > 20\
        order by Count(*) desc\
        limit 1000;

Done.


In [94]:
def gender(l):
    if l == 'M':
        return 'male'
    else:
        return 'female'


cohort_patients = [Beneficiary(hicno=row[0], sex=gender(row[1]), dob=str(row[2])) for row in cohort_study[1:]]


In [150]:
hicno = [pat.hicno for pat in cohort_patients]
hicno = tuple(hicno)
diags = %sql SELECT DISTINCT PatientID, Diagnosis from diagnoses where PatientID in $hicno
diags = diags.DataFrame().set_index('PatientId')

for pat in cohort_patients[0:3]:
    pat_diags = diags['Diagnosis'].loc[pat.hicno].tolist()
    for code in pat_diags:
        pat.add_diagnosis(Diagnosis(pat,code,ICDType.NINE))
        

#jane.add_diagnosis(Diagnosis(jane,code, int(ICDType.NINE)))

Done.


In [152]:
cohort_score  = [score(pat,"community",Score) for pat in cohort_patients]

In [ ]:

cohort_score
